Lung Fibrosis Training notebook for MLP, Quantile Regression and Graident Boost Tree algorithm

# MLP

In [ ]:
    #import library
    import numpy as np
    import pandas as pd
    import pydicom
    import os
    import random
    import matplotlib.pyplot as plt
    from tqdm import tqdm
    from PIL import Image
    from sklearn.metrics import mean_absolute_error
    from sklearn.model_selection import KFold

    import tensorflow as tf
    import tensorflow.keras.backend as K
    import tensorflow.keras.layers as L
    import tensorflow.keras.models as M
    #import autokeras as ak

In [ ]:
#set seeding
def seed_everything(seed=2020):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(22)

In [ ]:
#set root path
ROOT = "../input/osic-pulmonary-fibrosis-progression"
#set batch size
BATCH_SIZE=128

In [ ]:
#read in train and test
tr = pd.read_csv(f"{ROOT}/train.csv")
tr.drop_duplicates(keep=False, inplace=True, subset=['Patient','Weeks'])
chunk = pd.read_csv(f"{ROOT}/test.csv")

# read in sample submission file
print("add infos")
sub = pd.read_csv(f"{ROOT}/sample_submission.csv")
sub['Patient'] = sub['Patient_Week'].apply(lambda x:x.split('_')[0])
sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
sub =  sub[['Patient','Weeks','Confidence','Patient_Week']]
sub = sub.merge(chunk.drop('Weeks', axis=1), on="Patient")

In [ ]:
tr.head(5)

In [ ]:
chunk.head(5)

In [ ]:
sub.head(5)

In [ ]:
#combine into one dataframe for preprocessing
tr['WHERE'] = 'train'
chunk['WHERE'] = 'val'
sub['WHERE'] = 'test'
data = tr.append([chunk])

In [ ]:
data

In [ ]:
#print out shape for train, validation and test and number of unique record
print(tr.shape, chunk.shape, data.shape)
print(tr.Patient.nunique(), chunk.Patient.nunique(), 
      data.Patient.nunique())

In [ ]:
#calcuate the minimal week for each patient, given there are multiple weeks of scan for each patient, feature engineering
data['min_week'] = data['Weeks']
data.loc[data.WHERE=='test','min_week'] = np.nan
data['min_week'] = data.groupby('Patient')['min_week'].transform('min')

In [ ]:
data.head(5)

In [ ]:
#filter out patient that have only 1 week of FCV

base = data.loc[data.Weeks == data.min_week]
base = base[['Patient','FVC']].copy()
base.columns = ['Patient','min_FVC']
base['nb'] = 1
base['nb'] = base.groupby('Patient')['nb'].transform('cumsum')
base = base[base.nb==1]
base.drop('nb', axis=1, inplace=True)

In [ ]:
base.head(5)

In [ ]:
#create data frame that show min and current week & FCV for each patient, dups exist
data = data.merge(base, on='Patient', how='left')
data['base_week'] = data['Weeks'] - data['min_week']
del base

In [ ]:
data.head(5)

In [ ]:
#creating dummy variable for categorical variables

COLS = ['Sex','SmokingStatus'] #,'Age'
FE = []
for col in COLS:
    for mod in data[col].unique():
        FE.append(mod)
        data[mod] = (data[col] == mod).astype(int)
#=================

In [ ]:
data.head(5)

In [ ]:
# normalize the numerical values using min-max 

data['age'] = (data['Age'] - data['Age'].min() ) / ( data['Age'].max() - data['Age'].min() )
data['BASE'] = (data['min_FVC'] - data['min_FVC'].min() ) / ( data['min_FVC'].max() - data['min_FVC'].min() )
data['week'] = (data['base_week'] - data['base_week'].min() ) / ( data['base_week'].max() - data['base_week'].min() )
data['percent'] = (data['Percent'] - data['Percent'].min() ) / ( data['Percent'].max() - data['Percent'].min() )
FE += ['age','percent','week','BASE']

In [ ]:
FE

In [ ]:
data.head(5)

In [ ]:
y = data.FVC
x = data.drop(['Patient','Weeks','Percent','Age','Sex','SmokingStatus','WHERE','min_week','base_week','FVC'],axis=1)

In [ ]:
x.head(5)

In [ ]:
#break aprat to train, validation and test
#tr = data.loc[data.WHERE=='train']
#chunk = data.loc[data.WHERE=='val']
#sub = data.loc[data.WHERE=='test']
#del data

In [ ]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
xTrain.shape, xTest.shape,yTrain.shape, yTest.shape 

In [ ]:
#tr.shape, chunk.shape, sub.shape

In [ ]:
yTrain.values

### BASELINE NN 

In [ ]:
C1, C2 = tf.constant(70, dtype='float32'), tf.constant(1000, dtype="float32")
#=============================#
def score(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return K.mean(metric)
#============================#
def qloss(y_true, y_pred):
    # Pinball loss for multiple quantiles
    qs = [0.2, 0.50, 0.8]
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q*e, (q-1)*e)
    return K.mean(v)
#=============================#
def mloss(_lambda):
    def loss(y_true, y_pred):
        return _lambda * qloss(y_true, y_pred) + (1 - _lambda)*score(y_true, y_pred)
    return loss
#=================
def make_model(nh):
    z = L.Input((nh,), name="Patient")
    x = L.Dense(1000, activation="relu", name="d1")(z)
    x = L.Dense(1000, activation="relu", name="d2")(x)
    #x = L.BatchNormalization()(x)
    #x = L.Dense(1000, activation="relu", name="d3")(x)
    #x = L.BatchNormalization()(x)
    #x = L.Dropout(.3)(x)
    
    #x = L.Dense(1000, activation="relu", name="d3")(x)
    p1 = L.Dense(3, activation="linear", name="p1")(x)
    p2 = L.Dense(3, activation="relu", name="p2")(x)
    preds = L.Lambda(lambda x: x[0] + tf.cumsum(x[1], axis=1), 
                     name="preds")([p1, p2])
    
    model = M.Model(z, preds, name="CNN")
    #model.compile(loss=qloss, optimizer="adam", metrics=[score])
    model.compile(loss=mloss(0.8), optimizer=tf.keras.optimizers.Adam(lr=0.02, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False), metrics=[score])
    return model

In [ ]:
#y1 = tr['FVC'].values
#z1 = tr[FE].values
#ze = sub[FE].values
#nh = z.shape[1]
#pe = np.zeros((ze.shape[0], 3))
#pred = np.zeros((z.shape[0], 3))

In [ ]:
y = y.values
z = x.values
#ze = sub[FE].values
nh = z.shape[1]
#pe = np.zeros((ze.shape[0], 3))
pred = np.zeros((z.shape[0], 3))

In [ ]:
y = yTrain.values
z = xTrain.values
#ze = sub[FE].values
nh = z.shape[1]
#pe = np.zeros((ze.shape[0], 3))
pred = np.zeros((z.shape[0], 3))

In [ ]:
net = make_model(nh)
print(net.summary())
print(net.count_params())

In [ ]:
NFOLD = 5
kf = KFold(n_splits=NFOLD)

In [ ]:
print(z.shape)
print(y.shape)

In [ ]:
history = net.fit(z, y, validation_split=0.2, epochs=10, batch_size=128, verbose=1)

In [ ]:
print(history.history.keys())

In [ ]:
print(history.history)

In [ ]:
plt.plot(history.history['score'])
plt.plot(history.history['val_score'])
plt.title('model score')
plt.ylabel('score')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
%%time
cnt = 0
EPOCHS = 1000
for tr_idx, val_idx in kf.split(z):
    cnt += 1
    print(f"FOLD {cnt}")
    net = make_model(nh)
    net.fit(z[tr_idx], y[tr_idx], batch_size=BATCH_SIZE, epochs=EPOCHS, 
            validation_data=(z[val_idx], y[val_idx]), verbose=0) #
    print("train", net.evaluate(z[tr_idx], y[tr_idx], verbose=0, batch_size=BATCH_SIZE))
    print("val", net.evaluate(z[val_idx], y[val_idx], verbose=0, batch_size=BATCH_SIZE))
    print("predict val...")
    pred[val_idx] = net.predict(z[val_idx], batch_size=BATCH_SIZE, verbose=0)
    print("predict test...")
    #pe += net.predict(ze, batch_size=BATCH_SIZE, verbose=0) / NFOLD
#==============

In [ ]:
#predict validation test
predy = net.predict(z,verbose=0)

In [ ]:
predy

In [ ]:
#sigma_opt = mean_absolute_error(y, pred[:, 1])
#unc = pred[:,2] - pred[:, 0]
#sigma_mean = np.mean(unc)
print(sigma_opt, sigma_mean)

In [ ]:
idxs = np.random.randint(0, y.shape[0], 100)
plt.plot(y[idxs], label="ground truth")
plt.plot(pred[idxs, 0], label="q25")
#plt.plot(pred[idxs, 1], label="q50")
#plt.plot(pred[idxs, 2], label="q75")#plt.legend(loc="best")
plt.show()

In [ ]:
#print(unc.min(), unc.mean(), unc.max(), (unc>=0).mean())

In [ ]:
plt.hist(unc)
plt.title("uncertainty in prediction")
plt.show()

In [ ]:
#train['ypredL'] = modelL.predict( train ).values
#train['ypred']  = model.predict( train ).values
#train['ypredH'] = modelH.predict( train ).values


#train['ypredstd'] = 0.5*np.abs(train['ypredH'] - train['ypred'])+0.5*np.abs(train['ypred'] - train['ypredL'])
#train.head(10)


#predictstd = 0.5*np.abs(pred[val_idx][:,2] - pred[val_idx][:,1])+0.5*np.abs(pred[val_idx][:,1] - pred[val_idx][:,0])

predictstdy = 0.5*np.abs(predy[:,2] - predy[:,1])+0.5*np.abs(predy[:,1] - predy[:,0])

In [ ]:
def metric( trueFVC, predFVC, predSTD ):
    
    clipSTD = np.clip( predSTD, 70 , 9e9 )  
    
    deltaFVC = np.clip( np.abs(trueFVC-predFVC), 0 , 1000 )  

    return np.mean( -1*(np.sqrt(2)*deltaFVC/clipSTD) - np.log( np.sqrt(2)*clipSTD ) )
    

print( 'Metric:', metric(yTest.values,  predy[:,1], predictstdy))

### PREDICTION

In [ ]:
sub.head()

In [ ]:
sub['FVC1'] = 0.996*pe[:, 1]
sub['Confidence1'] = pe[:, 2] - pe[:, 0]

In [ ]:
subm = sub[['Patient_Week','FVC','Confidence','FVC1','Confidence1']].copy()

In [ ]:
subm.loc[~subm.FVC1.isnull()].head(10)

In [ ]:
subm.loc[~subm.FVC1.isnull(),'FVC'] = subm.loc[~subm.FVC1.isnull(),'FVC1']
if sigma_mean<70:
    subm['Confidence'] = sigma_opt
else:
    subm.loc[~subm.FVC1.isnull(),'Confidence'] = subm.loc[~subm.FVC1.isnull(),'Confidence1']

In [ ]:
subm.head()

In [ ]:
subm.describe().T

In [ ]:
otest = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')
for i in range(len(otest)):
    subm.loc[subm['Patient_Week']==otest.Patient[i]+'_'+str(otest.Weeks[i]), 'FVC'] = otest.FVC[i]
    subm.loc[subm['Patient_Week']==otest.Patient[i]+'_'+str(otest.Weeks[i]), 'Confidence'] = 0.1

In [ ]:
subm[["Patient_Week","FVC","Confidence"]].to_csv("submission.csv", index=False)

# Quantile regression

In [ ]:
#quantile regression

from statsmodels.formula.api import quantreg
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
train = pd.read_csv( '../input/osic-pulmonary-fibrosis-progression/train.csv' )
test  = pd.read_csv( '../input/osic-pulmonary-fibrosis-progression/test.csv' )

train['traintest'] = 0
test ['traintest'] = 1

sub   = pd.read_csv( '../input/osic-pulmonary-fibrosis-progression/sample_submission.csv' )
sub['Weeks']   = sub['Patient_Week'].apply( lambda x: int(x.split('_')[-1]) )
sub['Patient'] = sub['Patient_Week'].apply( lambda x: x.split('_')[0] ) 

print( train.shape, test.shape, sub.shape )

#sub.head()

In [ ]:
train.Patient.nunique(), sub.Patient.nunique()

In [ ]:
train = pd.concat( (train,test) )
train.sort_values( ['Patient','Weeks'], inplace=True )
train.shape

In [ ]:
train.head(5)

In [ ]:
#calcuate the minimal week for each patient, given there are multiple weeks of scan for each patient, feature engineering
train['min_week'] = train['Weeks']
#train.loc[data.WHERE=='test','min_week'] = np.nan
train['min_week'] = train.groupby('Patient')['min_week'].transform('min')

In [ ]:
train.head(5)

In [ ]:
#filter out patient that have only 1 week of FCV

base = train.loc[train.Weeks == train.min_week]
base = base[['Patient','FVC']].copy()
base.columns = ['Patient','min_FVC']
base['nb'] = 1
base['nb'] = base.groupby('Patient')['nb'].transform('cumsum')
base = base[base.nb==1]
base.drop('nb', axis=1, inplace=True)

In [ ]:
base.head(5)

In [ ]:
#create data frame that show min and current week & FCV for each patient, dups exist
data = train.merge(base, on='Patient', how='left')
data['base_week'] = data['Weeks'] - data['min_week']
del base

In [ ]:
data.head(5)

In [ ]:
data.describe()

In [ ]:
data.groupby( ['Sex','SmokingStatus'] )['FVC'].agg( ['mean','std','count'] )

In [ ]:
#label enconding

data['Sex']           = pd.factorize( data['Sex'] )[0]
data['SmokingStatus'] = pd.factorize( data['SmokingStatus'] )[0]
data.head(5)

In [ ]:
#creating z score

data['Percent']       = (data['Percent'] - data['Percent'].mean()) / data['Percent'].std()
data['Age']           = (data['Age'] - data['Age'].mean()) / data['Age'].std()
data['Sex']           = (data['Sex'] - data['Sex'].mean()) / data['Sex'].std()
data['SmokingStatus'] = (data['SmokingStatus'] - data['SmokingStatus'].mean()) / data['SmokingStatus'].std()
data.head(10)

In [ ]:
data.head(5)

In [ ]:
y = data.FVC
x = data.drop(['Patient','traintest','base_week'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
xTrain.head(5)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

corrmat = xTrain.corr() 
f, ax = plt.subplots(figsize =(9, 8)) 
sns.heatmap(corrmat, ax = ax, cmap = 'RdYlBu_r', linewidths = 0.5)

In [ ]:
xTrain.head(5)

In [ ]:
modelL = quantreg('FVC ~ Weeks+Age+Sex+SmokingStatus+min_FVC+Percent+min_week+min_FVC', xTrain).fit( q=0.15 )
model  = quantreg('FVC ~ Weeks+Age+Sex+SmokingStatus+min_FVC+Percent+min_week+min_FVC', xTrain).fit( q=0.50 )
modelH = quantreg('FVC ~ Weeks+Age+Sex+SmokingStatus+min_FVC+Percent+min_week+min_FVC', xTrain).fit( q=0.85 )
print(model.summary())

In [ ]:
yTrain.head(5)

In [ ]:
xTest['ypredL'] = modelL.predict( xTest ).values
xTest['ypred']  = model.predict( xTest ).values
xTest['ypredH'] = modelH.predict( xTest ).values
xTest['ypredstd'] = 0.5*np.abs(xTest['ypredH'] - xTest['ypred'])+0.5*np.abs(xTest['ypred'] - xTest['ypredL'])
xTest.head(10)

In [ ]:
#modelL = quantreg('FVC ~ Weeks+Age+Sex+SmokingStatus', train).fit( q=0.15 )
#model  = quantreg('FVC ~ Weeks+Age+Sex+SmokingStatus', train).fit( q=0.50 )
#modelH = quantreg('FVC ~ Weeks+Age+Sex+SmokingStatus', train).fit( q=0.85 )
#print(model.summary())

In [ ]:
#train['ypredL'] = modelL.predict( train ).values
#train['ypred']  = model.predict( train ).values
#train['ypredH'] = modelH.predict( train ).values
#train['ypredstd'] = 0.5*np.abs(train['ypredH'] - train['ypred'])+0.5*np.abs(train['ypred'] - train['ypredL'])
#train.head(10)

In [ ]:
def metric( trueFVC, predFVC, predSTD ):
    
    clipSTD = np.clip( predSTD, 70 , 9e9 )  
    
    deltaFVC = np.clip( np.abs(trueFVC-predFVC), 0 , 1000 )  

    return np.mean( -1*(np.sqrt(2)*deltaFVC/clipSTD) - np.log( np.sqrt(2)*clipSTD ) )
    

print( 'Metric:', metric( xTest['FVC'].values, xTest['ypred'].values, xTest['ypredstd'].values  ) )

In [ ]:
y = train['FVC'].values
ypredL = train['ypredL'].values
ypred = train['ypred'].values
ypredH = train['ypredH'].values

idxs = np.random.randint(0, y.shape[0], 100)
plt.plot(y[idxs], label="ground truth")
plt.plot(ypredL[idxs], label="q25")
plt.plot(ypred[idxs], label="q50")
plt.plot(ypredH[idxs], label="q75")
plt.legend(loc="best")
plt.show()

In [ ]:
sigma_opt_qr = mean_absolute_error(y, ypred)
unc_qr = ypredH - ypredL
sigma_mean_qr = np.mean(unc_qr)
print(sigma_opt_qr, sigma_mean_qr)

In [ ]:
plt.hist(unc_qr)
plt.title("uncertainty in prediction")
plt.show()

# GBM Quantile Regression

In [ ]:
#GBM regression
data.head(5)

In [ ]:
y = data.FVC
x = data.drop(['FVC','Patient'],axis=1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100,
                           n_jobs = -1,
                           oob_score = True,
                           bootstrap = True,
                           random_state = 42)
rf.fit(x, y)

In [ ]:
# udfs ----

# function for creating a feature importance dataframe
def imp_df(column_names, importances):
    df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
    return df

# plotting a feature importance dataframe (horizontal barchart)
def var_imp_plot(imp_df, title):
    imp_df.columns = ['feature', 'feature_importance']
    sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df, orient = 'h', color = 'royalblue') \
       .set_title(title, fontsize = 20)


In [ ]:

base_imp = imp_df(x.columns, rf.feature_importances_)
base_imp

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

var_imp_plot(base_imp, 'Default feature importance (scikit-learn)')

In [ ]:
#x = train[["Weeks","Percent","Age","Sex","SmokingStatus"]]
#y = train[["FVC"]]

In [ ]:
# import machine learning libraries
import xgboost as xgb
from sklearn.metrics import accuracy_score


# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

In [ ]:
from sklearn.metrics.regression import mean_absolute_error as mae
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
import numpy as np

def objective_function_regression(estimator):
    mae_array = cross_val_score( estimator, X_train, y_train, cv= 3, n_jobs=-1, scoring = make_scorer(mae) )
    return np.mean(mae_array)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from hyperopt import hp, fmin, tpe
from hyperopt.pyll import scope

# hyperopt object for 
scope.define(GradientBoostingRegressor)

# search space
n_estimators  = hp.randint('n_estimators',1000) 
learning_rate = hp.loguniform('learning_rate',-3,1)
max_depth     = hp.randint('max_depth', 10)
max_features =  hp.randint('max_features',x.shape[1]-1)
min_samples_leaf = hp.randint('min_samples_leaf', 10)
min_samples_spilt = hp.randint('min_samples_leaf', 10)
                                       
# model / estimator to be optimized
est0 = (0.1, scope.GradientBoostingRegressor( n_estimators  = n_estimators + 1,
                                            learning_rate = learning_rate,
                                            max_depth = max_depth + 1,
                                            max_features = max_features + 1,
                                            min_samples_leaf = min_samples_leaf + 1,
                                            min_samples_spilt = min_samples_spilt + 1,                                           
                                            random_state=42) 
        )

# search space
search_space_regression = hp.pchoice('estimator', [est0])

In [ ]:
best = fmin(
    fn= objective_function_regression,
    space= search_space_regression,
    algo = tpe.suggest, # This is the optimization algorithm hyperopt uses, a tree of parzen estimators
    max_evals = 1000,
    verbose = 1  # The number of iterations
         )

print(best)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
quantiles = [0.15, 0.50, 0.85]


# Get the model and the predictions in (a) - (b)
def GBM(q):
    
   # (a) Modeling  500
   mod = GradientBoostingRegressor(loss='quantile', alpha=q,
                                n_estimators=1000, max_depth=5,max_features = 3,
                                learning_rate=0.05599876950226799, min_samples_leaf=6,
                                min_samples_split=20)
   mod.fit(X_train, y_train)

   # (b) Predictions
   pred = pd.Series(mod.predict(X_test).round(2))
   return pred, mod

GBM_models=[]
GBM_actual_pred = pd.DataFrame()

for q in quantiles:
    pred , model = GBM(q)
    GBM_models.append(model)
    GBM_actual_pred = pd.concat([GBM_actual_pred,pred],axis=1)
    
GBM_actual_pred.columns=quantiles
#GBM_actual_pred['actual'] = y
#GBM_actual_pred['interval'] = GBM_actual_pred[np.max(quantiles)] - GBM_actual_pred[np.min(quantiles)]
#GBM_actual_pred = GBM_actual_pred.sort_values('interval')

In [ ]:
GBM_actual_pred['actual'] = y_test.values

In [ ]:
GBM_actual_pred['interval'] = GBM_actual_pred[np.max(quantiles)] - GBM_actual_pred[np.min(quantiles)]

In [ ]:
GBM_actual_pred

In [ ]:
GBM_actual_pred = GBM_actual_pred.rename(columns={0.15: 'ypredL', 0.5: 'ypred',0.85: 'ypredH'})

In [ ]:
GBM_actual_pred['ypredstd'] = 0.5*np.abs(GBM_actual_pred['ypredH'] - GBM_actual_pred['ypred'])+0.5*np.abs(GBM_actual_pred['ypred'] - GBM_actual_pred['ypredL'])

In [ ]:
def metric( trueFVC, predFVC, predSTD ):
    
    clipSTD = np.clip( predSTD, 70 , 9e9 )  
    
    deltaFVC = np.clip( np.abs(trueFVC-predFVC), 0 , 1000 )  

    return np.mean( -1*(np.sqrt(2)*deltaFVC/clipSTD) - np.log( np.sqrt(2)*clipSTD ) )
    

print( 'Metric:', metric( GBM_actual_pred['actual'].values, GBM_actual_pred['ypred'].values, GBM_actual_pred['ypredstd'].values  ) )

In [ ]:
y = GBM_actual_pred['actual'].values
ypredL = GBM_actual_pred['ypredL'].values
ypred = GBM_actual_pred['ypred'].values
ypredH = GBM_actual_pred['ypredH'].values

idxs = np.random.randint(0, y.shape[0], 100)
plt.plot(y[idxs], label="ground truth")
plt.plot(ypredL[idxs], label="q25")
plt.plot(ypred[idxs], label="q50")
plt.plot(ypredH[idxs], label="q75")
plt.legend(loc="best")
plt.show()

In [ ]:
sigma_opt_qr = mean_absolute_error(y, ypred)
unc_qr = ypredH - ypredL
sigma_mean_qr = np.mean(unc_qr)
print(sigma_opt_qr, sigma_mean_qr)

In [ ]:
plt.hist(unc_qr)
plt.title("uncertainty in prediction")
plt.show()